In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

C:\Users\junta\Anaconda2\envs\py3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Epoch 0 MSE = 2.7544272
Epoch 100 MSE = 0.6322219
Epoch 200 MSE = 0.57277966
Epoch 300 MSE = 0.5585005
Epoch 400 MSE = 0.54906934
Epoch 500 MSE = 0.5422877
Epoch 600 MSE = 0.5373788
Epoch 700 MSE = 0.5338219
Epoch 800 MSE = 0.5312427
Epoch 900 MSE = 0.52937055
Best theta:
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845096e-01]
 [ 1.64778158e-01]
 [ 7.44080753e-04]
 [-3.91945168e-02]
 [-8.61356616e-01]
 [-8.23479712e-01]]


In [2]:
graph = tf.get_default_graph()
operations = graph.get_operations()

In [3]:
graph

In [4]:
operations

[<tf.Operation 'X' type=Const>,
 <tf.Operation 'y' type=Const>,
 <tf.Operation 'random_uniform/shape' type=Const>,
 <tf.Operation 'random_uniform/min' type=Const>,
 <tf.Operation 'random_uniform/max' type=Const>,
 <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'random_uniform/sub' type=Sub>,
 <tf.Operation 'random_uniform/mul' type=Mul>,
 <tf.Operation 'random_uniform' type=Add>,
 <tf.Operation 'theta' type=VariableV2>,
 <tf.Operation 'theta/Assign' type=Assign>,
 <tf.Operation 'theta/read' type=Identity>,
 <tf.Operation 'predictions' type=MatMul>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'Square' type=Square>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'mse' type=Mean>,
 <tf.Operation 'transpose/Rank' type=Rank>,
 <tf.Operation 'transpose/sub/y' type=Const>,
 <tf.Operation 'transpose/sub' type=Sub>,
 <tf.Operation 'transpose/Range/start' type=Const>,
 <tf.Operation 'transpose/Range/delta' type=Const>,
 <tf.Operation 'transpose/Range' typ

In [5]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [6]:
show_graph(graph)

In [7]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.7544272
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727796
Epoch 300 MSE = 0.5585005
Epoch 400 MSE = 0.54906934
Epoch 500 MSE = 0.5422877
Epoch 600 MSE = 0.5373788
Epoch 700 MSE = 0.5338219
Epoch 800 MSE = 0.5312427
Epoch 900 MSE = 0.52937055
Best theta:
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


In [8]:
graph = tf.get_default_graph()
operations = graph.get_operations()

In [9]:
graph

In [10]:
operations

[<tf.Operation 'X' type=Const>,
 <tf.Operation 'y' type=Const>,
 <tf.Operation 'random_uniform/shape' type=Const>,
 <tf.Operation 'random_uniform/min' type=Const>,
 <tf.Operation 'random_uniform/max' type=Const>,
 <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'random_uniform/sub' type=Sub>,
 <tf.Operation 'random_uniform/mul' type=Mul>,
 <tf.Operation 'random_uniform' type=Add>,
 <tf.Operation 'theta' type=VariableV2>,
 <tf.Operation 'theta/Assign' type=Assign>,
 <tf.Operation 'theta/read' type=Identity>,
 <tf.Operation 'predictions' type=MatMul>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'Square' type=Square>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'mse' type=Mean>,
 <tf.Operation 'gradients/Shape' type=Const>,
 <tf.Operation 'gradients/grad_ys_0' type=Const>,
 <tf.Operation 'gradients/Fill' type=Fill>,
 <tf.Operation 'gradients/mse_grad/Reshape/shape' type=Const>,
 <tf.Operation 'gradients/mse_grad/Reshape' type=Reshape>,
 <tf.Opera

In [11]:
show_graph(graph)